# CityShadows: Mapping Building Heights
This notebook processes a JSON file with the following fields:

- fileName – Latitude, longitude, and degrees (lat,long_deg).
- objectType – Either "structure" or "vegetation".
- distance – Object distance from the camera (float).
- height – Estimated height of the object (float).
- width – Estimated width of the object (float).
- horizontalOffset – Horizontal position of the object in the image (-100 = leftmost, 0 = center, 100 = rightmost).
- isCut – Array indicating whether the object is partially cut off in the image.

The script matches each estimated height to the appropriate building footprint, using building geometries from a shapefile.

For this study, we used a subset of **Makati City, Metro Manila**. The building footprints were obtained from https://www.geofabrik.de/.

## Libraries

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import json
from shapely.geometry import Point, Polygon
from shapely.affinity import rotate

## Variables

### shp files

In [ ]:
small_makati = gpd.read_file("../../data/smaller_makati.shp")
buildings = gpd.read_file("../../data/gis_osm_buildings_a_free_1.shp")
less_makati_buildings = buildings[small_makati['geometry'].item().contains(buildings['geometry'])]

### json

In [ ]:
with open("output-updated.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)

structures_df = df[df['objectType'] == 'structure'].copy()

# Remove entries with heights < 2 m or > 280 m
structures_df = structures_df[(structures_df['height'] >= 2) & (structures_df['height'] <= 280)]

print(f"Found {len(structures_df)} structures after filtering")
print(f"Unique files: {structures_df['fileName'].nunique()}")
structures_df.head()

## Function Declaration
### Calculate building position

In [ ]:
def calculate_building_position(
    lat: float,
    lon: float,
    bearing: float,
    distance: float = None,              
    horizontal_offset: float = 0.0,       
    pitch_degrees: float = 20.0,        
    fov_degrees: float = 120.0,          
    camera_height: float = 2.45,          
    structure_width: float = None,        
    use_pitch_only: bool = False          
) -> Polygon:

    # Convert lat/lon to planar (meters)
    camera_point = gpd.GeoSeries([Point(lon, lat)], crs="EPSG:4326")
    camera_xy = camera_point.to_crs(epsg=3857).iloc[0]

    # Convert horizontal image offset to angular adjustment
    ratio = horizontal_offset / 100.0
    half_fov_rad = math.radians(fov_degrees / 2.0)
    angle_offset = math.degrees(math.atan(ratio * math.tan(half_fov_rad)))
    eff_bearing = bearing + angle_offset

    # Pitch -> ground distance
    pitch_rad = math.radians(-pitch_degrees)

    if use_pitch_only:
        ground_distance = camera_height / math.tan(pitch_rad)
    else:
        if distance is None:
            raise ValueError("Slant distance required when use_pitch_only=False.")
        ground_distance = distance * math.cos(pitch_rad)

    # Bearing to delta-x/y
    rad = math.radians(eff_bearing)
    dx = ground_distance * math.sin(rad)
    dy = ground_distance * math.cos(rad)

    # Final ground position
    center_x = camera_xy.x + dx
    center_y = camera_xy.y + dy
    center_point = Point(center_x, center_y)

    if structure_width is None:
        return center_point

    # Build axis-aligned square around center
    half_w = structure_width / 2.0
    square = Polygon([
        (center_x - half_w, center_y - half_w),
        (center_x + half_w, center_y - half_w),
        (center_x + half_w, center_y + half_w),
        (center_x - half_w, center_y + half_w),
    ])

    # Rotate to match direction of structure
    rotated = rotate(square, eff_bearing, origin=center_point, use_radians=False)
    return rotated

## Prep buildings GeoDataFrame for GeoJSON export

In [ ]:
buildings_with_height = less_makati_buildings.copy()
buildings_with_height['height'] = np.nan
buildings_with_height

## Process all buildings and map to appropriate OSM ID

Estimate building distance from camera, create a square polygon and the footprint with most intersection is assigned the height. If more than one assigned height, get the mean.

In [ ]:
max_match_dist = 20  
less_makati_proj = less_makati_buildings.to_crs(epsg=3857)

for col in ['source_fileName', 'horizontal_offset', 'is_cut']:
    if col not in buildings_with_height.columns:
        buildings_with_height[col] = None

height_obs = {}

for idx, row in structures_df.iterrows():
    try:
        # Parse camera lat/lon and viewing angle
        lat, lon = map(float, row['fileName'].split('_')[0].split(','))
        angle = float(row['fileName'].split('_')[1])

        # Compute rotated footprint polygon
        footprint = calculate_building_position(
            lat=lat,
            lon=lon,
            bearing=angle,
            distance=row['distance'],
            horizontal_offset=row['horizontalOffset'],
            structure_width=row['width'],
            camera_height=2.45,
            use_pitch_only=False
        )

        # Try polygon‐based match
        det_gdf = gpd.GeoDataFrame(
            {'height_det': [row['height']], 'geometry': [footprint]},
            crs=less_makati_proj.crs
        )
        inter = gpd.overlay(det_gdf, less_makati_proj, how='intersection')

        osm_id = None
        if not inter.empty:
            inter['area'] = inter.geometry.area
            best = inter.loc[inter['area'].idxmax()]
            # check distance cap from structure centroid to matched building
            if best.geometry.distance(footprint.centroid) <= max_match_dist:
                osm_id = best['osm_id']

        # Fallback: nearest building centroid to footprint centroid
        if osm_id is None:
            center = footprint.centroid
            dists = less_makati_proj.geometry.distance(center)
            valid = dists[dists <= max_match_dist]
            if valid.empty:
                print(f"Structure #{idx+1}: no match within {max_match_dist}m → skipped")
                continue
            nearest_idx = valid.idxmin()
            osm_id = less_makati_proj.at[nearest_idx, 'osm_id']

        # Append height to structure
        height_obs.setdefault(osm_id, []).append(row['height'])
        # median_h = np.median(height_obs[osm_id])
        mean_h = np.mean(height_obs[osm_id])


        # Update building record
        sel = buildings_with_height['osm_id'] == osm_id
        # buildings_with_height.loc[sel, 'height'] = median_h
        buildings_with_height.loc[sel, 'height'] = mean_h
        buildings_with_height.loc[sel, 'source_fileName'] = (
            buildings_with_height.loc[sel, 'source_fileName'].fillna('') + ";" + row['fileName']
        )
        buildings_with_height.loc[sel, 'horizontal_offset'] = row['horizontalOffset']
        buildings_with_height.loc[sel, 'is_cut'] = (
            ','.join(row['isCut']) if isinstance(row['isCut'], list) else str(row['isCut'])
        )

        # print(f"Structure #{idx+1} -> OSM {osm_id}: median height={median_h:.2f} m")
        print(f"Structure #{idx+1} -> OSM {osm_id}: mean height={mean_h:.2f} m")

    except Exception as e:
        print(f"[ERROR] Structure #{idx+1} → {e}")
        continue

# Summary
print(f"\nTotal structures processed:   {len(structures_df)}")
print(f"Unique buildings seen:          {len(height_obs)}")
print(f"Buildings with height data:     {buildings_with_height['height'].notna().sum()}")

## Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

less_makati_buildings.plot(ax=ax1, color='lightgray', edgecolor='black', alpha=0.7)
buildings_with_height[buildings_with_height['height'].notna()].plot(
    ax=ax1, color='red', edgecolor='darkred', alpha=0.8
)
ax1.set_title('Buildings with Height Data (Red = Matched Buildings)')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')

buildings_with_height_only = buildings_with_height[buildings_with_height['height'].notna()]
if not buildings_with_height_only.empty:
    buildings_with_height_only.plot(
        ax=ax2, column='height', cmap='viridis', legend=True, 
        edgecolor='black', alpha=0.8
    )
    ax2.set_title('Building Heights (Colored by Height)')
    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Latitude')

plt.tight_layout()
plt.savefig("building_heights.png", dpi=300) 
plt.show()

## Export GDF to GeoJSON file

In [ ]:
buildings_with_height.to_file("buildings_dataset.geojson", driver="GeoJSON")